# 3.2.3. T-distributed Stochastic Neighbor Embedding

## Preparación del Entorno

### Carga de Módulos

In [ ]:
import math
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
import session_info
from time import time
from plotly.subplots import make_subplots
from sklearn import set_config
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import sys

sys.path.append('../scripts')

# Tema Principal
from sklearn.manifold import TSNE
from funny_stuffs import generate_contrast_colors, plot_embedding

In [ ]:
session_info.show()

### Configuración Inicial

In [ ]:
random_seed = 333  # Semilla para reproducibilidad de resultados
np.random.seed(random_seed)  # Para reproducibilidad

# Configuración de opciones de visualización para pandas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas
pd.set_option('display.max_rows', 15)  # Ajusta el número de filas a mostrar

# Configuraciones extras
sns.set_style('dark')
dark_template = pio.templates['plotly_dark'].to_plotly_json()
dark_template['layout']['paper_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
dark_template['layout']['plot_bgcolor'] = 'rgba(30, 30, 30, 0.5)'
pio.templates['plotly_dark_semi_transparent'] = go.layout.Template(dark_template)
pio.templates.default = 'plotly_dark_semi_transparent'
#set_config(transform_output="pandas")
set_config(display='diagram')
warnings.filterwarnings("ignore")
%matplotlib inline

## T-distributed Stochastic Neighbor Embedding

### Fundamento Teórico

La Incrustación Estocástica de Vecinos T-Describuidos (*t-SNE T-distributed Stochastic Neighbor Embedding* en inglés) es una técnica de reducción de dimensionalidad enfocada en la visualización de datos complejos, transformando distancias entre puntos en probabilidades para preservar la estructura de los datos al reducirlos a dos o tres dimensiones. Optimiza la ubicación de los puntos en el espacio reducido para que su distribución de probabilidades se asemeje a la del espacio original.

Para realizar el procedimiento, debemos de hacer lo siguiente:

#### 1. Obtener las Probabilidades Conjuntas en el Espacio de Alta Dimensión

Dado un conjunto de datos $\{x_1, x_2, ..., x_n\}$, t-SNE comienza calculando probabilidades conjuntas $p_{ij}$ que representan similitudes entre pares de puntos. La probabilidad conjunta $p_{ij}$ es simétrica y se define mediante la media de las probabilidades condicionales $p_{j|i}$ y $p_{i|j}$, donde $p_{j|i}$ se calcula como sigue:

$$
p_{j|i} = \frac{\exp(-||x_i - x_j||^2 / 2\sigma_i^2)}{\sum_{k \neq i}\exp(-||x_i - x_k||^2 / 2\sigma_i^2)}
$$

y la probabilidad conjunta $p_{ij}$ se define como:

$$
p_{ij} = \frac{p_{j|i} + p_{i|j}}{2n}
$$

La desviación estándar $\sigma_i$ para cada punto $x_i$ no es constante para todos los puntos; en cambio, se ajusta individualmente para cada punto de manera que la distribución de las probabilidades condicionales $p_{j|i}$ cumpla con un valor de perplejidad predefinido: 

$$
\text{Perplejidad} = 2^{H(P_i)}
$$

donde $H(P_i)$ es la entropía de Shannon de la distribución de probabilidad condicional $P_i$ y se calcula como:

$$
H(P_i) = -\sum_j p_{j|i} \log_2 p_{j|i}
$$

#### Proceso de Busqueda (Opcional)

In [ ]:
# ¿Cómo es el proceso de busqueda

# Datos de ejemplo
x1 = np.array([0, 0])
x2 = np.array([1, 0])
x3 = np.array([0, 1])

# Inicialización de sigma
sigma1 = 10
perplexity = 2.0
desired_entropy = np.log2(perplexity)

In [ ]:
# Función para calcular las probabilidades condicionales
def compute_probabilities(x_i, x_j, x_k, sigma_i):
    dist_ij = np.linalg.norm(x_i - x_j)
    dist_ik = np.linalg.norm(x_i - x_k)
    numerator = np.exp(-dist_ij**2 / (2 * sigma_i**2))
    denominator = np.exp(-dist_ij**2 / (2 * sigma_i**2)) + np.exp(-dist_ik**2 / (2 * sigma_i**2))
    p_ji = numerator / denominator
    return p_ji, 1 - p_ji

# Función para calcular la entropía de Shannon
def compute_entropy(p_ji, p_ki):
    return -p_ji * np.log2(p_ji) - p_ki * np.log2(p_ki)

In [ ]:
# Búsqueda de sigma_i utilizando búsqueda binaria
max_iter = 100
tol = 1e-5
lower_bound = 1e-10
upper_bound = 50.0

for _ in range(max_iter):
    p_2_1, p_3_1 = compute_probabilities(x1, x2, x3, sigma1)
    entropy = compute_entropy(p_2_1, p_3_1)
    if np.abs(entropy - desired_entropy) < tol:
        break
    if entropy < desired_entropy:
        lower_bound = sigma1
        sigma1 = (sigma1 + upper_bound) / 2
    else:
        upper_bound = sigma1
        sigma1 = (sigma1 + lower_bound) / 2


In [ ]:
sigma1, p_2_1, p_3_1, entropy

2.**Obtener las Probabilidades Conjuntas en el Espacio de Baja Dimensión**:

En el espacio de baja dimensión, buscamos encontrar los puntos $\{y_1, y_2, ..., y_n\}$ tal que las similitudes entre los puntos $q_{ij}$ reflejen las similitudes $p_{ij}$ del espacio de alta dimensión. La probabilidad conjunta $q_{ij}$ en el espacio de baja dimensión se define utilizando una **distribución t-Student con un grado de libertad**  como:

$$
q_{ij} = \frac{(1 + ||y_i - y_j||^2)^{-1}}{\sum_{k \neq l}(1 + ||y_k - y_l||^2)^{-1}}
$$

In [ ]:
# Efecto Crowding
normal_data = np.random.randn(1000)  # Distribución normal
student_data = np.random.standard_t(df=1, size=1000)  # Distribución t de Student con 1 grado de libertad

# Filtrar los datos de valores atípicos extremos
normal_data = normal_data[np.abs(normal_data) < 10]
student_data = student_data[np.abs(student_data) < 10]

fig = go.Figure()
fig.add_trace(go.Histogram(x=normal_data, nbinsx=50, name='Distribución Normal',
                           marker_color='skyblue', opacity=0.75))
fig.add_trace(go.Histogram(x=student_data, nbinsx=50, name='Distribución t-Student',
                           marker_color='orange', opacity=0.75))

fig.update_layout(title_text='Distribución Normal vs t-Student',
                  xaxis_title_text='Value', yaxis_title_text='Count',
                  bargap=0.2,
                  barmode='overlay',
                  xaxis_range=[-10, 10])

fig.show()

3.**Minimización de la Divergencia de Kullback-Leibler**:

El objetivo de t-SNE es minimizar la divergencia de Kullback-Leibler (KL) entre las distribuciones de probabilidad en los espacios de alta y baja dimensión, $P$ y $Q$ respectivamente. La divergencia KL se define como:

$$
C = KL(P||Q) = \sum_{i \neq j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
$$


4.**Optimización**:

La minimización de la divergencia de KL se realiza a través de técnicas de optimización, típicamente utilizando el método de **descenso de gradiente**. El gradiente de $C$ (función de costo) con respecto a cada punto $y_i$ en el espacio de baja dimensión se calcula como:

$$
\frac{\delta C}{\delta y_i} = 4 \sum_{j}(p_{ij} - q_{ij})(y_i - y_j)(1 + ||y_i - y_j||^2)^{-1}
$$

El descenso de gradiente actualiza los puntos $Y$ para minimizar $C$. La actualización se realiza de la siguiente manera:

$$
y_i^{(t+1)} = y_i^{(t)} - \eta \frac{\delta C}{\delta y_i}
$$

donde:

- $y_i^{(t)}$ es la posición del punto $y_i$ en la iteración $t$.
- $\eta$ es la tasa de aprendizaje.
- $\frac{\delta C}{\delta y_i}$ es el gradiente de la función de costo con respecto a $y_i$.
- 

El procedimiento de optimización ajusta iterativamente las posiciones $y_i$ en el espacio de baja dimensión para minimizar la divergencia KL, lo que resulta en una configuración de puntos que refleja fielmente las similitudes originales del espacio de alta dimensión.

#### Aplicaciónes y Limitaciones

<p style="font-size:25px;">Aplicaciones</p>

1. Visualización de Datos Multidimensionales
2. Exploración de Datos
3. Agrupación
4. Detección de Anomalías
5. Validación de Modelos


<p style="font-size:25px;">Limitaciones</p>

1. Costo Computacional
2. Sensibilidad a Hiperparámetros
3. Falta de Consistencia
4. No Preserva la Densidad Global
5. Dificultad en la Interpretación de Distancias
6. Tendencia al Overfitting
7. No es Determinístico

### Ejemplo Práctico

#### Preparación de los Datos

**Descripción del conjunto de datos:**

El conjunto de datos **Digits dataset**, es una colección de dígitos escritos a mano, contiene 1,797 imágenes en total.

Cada imagen es de `8x8` píxeles, lo que significa que cada imagen se representa como una matriz 2D de 8x8 o como un vector unidimensional de 64 píxeles al aplanarse. Estas están en escala de grises, con valores de píxeles que varían de 0 a 16. Esto representa la intensidad del píxel, donde un valor más alto corresponde a un píxel más oscuro.

Cada imagen viene con una etiqueta asociada que indica el dígito real que representa (entre 0 y 9).

In [ ]:
X, y = pd.read_pickle('../data/digits.pkl')
n_samples, n_features = X.shape
n_neighbors = 30

In [ ]:
#Muestra Dígitos
num_rows = 5
num_cols = 5

fig = make_subplots(
    rows=num_rows,
    cols=num_cols,
    subplot_titles=[None]*num_rows*num_cols)

for i in range(num_rows):
    for j in range(num_cols):
        index = i * num_cols + j
        image = X[index].reshape((8, 8))

        fig.add_trace(
            go.Heatmap(
                z=image,
                colorscale='Greys',
                showscale=False,
                hoverinfo='skip'
            ),
            row=i+1, col=j+1
        )

fig.update_layout(
    title_text='Conjunto de Dígitos',
    height=800,
    width=500,
    margin=dict(t=50, l=25, r=25, b=25),
    showlegend=False
)

fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)

fig.show()

#### Implementación del Método

Principales parámetros de t-SNE:
```python
TSNE(
    n_components=2,                # Parámetro Clave
    perplexity=30.0,               # Parámetro Clave
    early_exaggeration=12.0,       # Parámetro Clave
    learning_rate='auto',          # Parámetro Clave
    n_iter=1000,                   # Parámetro Clave
    n_iter_without_progress=300,
    min_grad_norm=1e-07,
    metric_params=None,
    init='pca',
    random_state=None,
)

#### Visualización e Implementación de Resultados

In [ ]:
reductor = {
    "PCA": PCA(n_components=2,
                        random_state=42),

    "t-SNE": TSNE(
        n_components=2,
        n_iter=500,
        perplexity = 30,
        n_iter_without_progress=150,
        random_state=random_seed
    ),
}

In [ ]:
projections, timing = {}, {}
for name, reducto in reductor.items():
    start_time = time()
    projections[name] = reducto.fit_transform(X, y)
    timing[name] = time() - start_time